In [1]:
import os, re
import pandas as pd
import numpy as np
import json

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline
plt.rcParams['font.size'] = 22

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from csw2022 import *

### reading from database: once stable move outside

'40 blocked': '1000clq',
'interleaved': 'i1000cl', 2018_.../exp_1020103153

In [3]:
def load_dfs(condition):
  ## load psiturk `experiment dataframe`
  sql_df = get_sql_df(get_dataset_code(condition))
  ## from `experiment dataframe`, pull data pertaining to subjects responses
  subj_df = make_subj_df(sql_df['datastring'].iloc[0])
  ## reindex dataframe to make easy to read
  group_df = make_group_df(sql_df)
  # breakup dataframe by story and question nodes
  _,_,(_,group_qdf),(_,group_sdf) = group_df.groupby('type')
  # breakup question dataframe by filler and transition nodes
  (_,group_fqdf),(_,group_tqdf) = group_qdf.groupby('qtype')
  return group_df,group_tqdf,group_sdf,group_qdf,group_fqdf

### dataframe processing

In [4]:
condition='interleaved_rep'
group_df,group_tqdf,group_sdf,group_qdf,group_fqdf = load_dfs(condition)

exp versions found in table:
10.13.18
1000cl
1000clq
9010cl
i1000cl
20B1000cl
RT01B1000cl
csw1000block40.04.07.19
csw1000block01.04.25.19
csw1000block02.04.26.19
csw1000rand50.04.29.19
csw1000block01intheloc.05.01.19
csw1000rand01.05.03.19
csw1000block02.05.07.19
csw1000boldmdp.05.09.19
csw1000insertedblock0.09.22.19
csw1000insertedblock1.09.30.19
csw1000insertedblock1.10.07.19
csw1000insertedblock0.10.08.19
csw1000insertedblock2.10.09.19
csw1000insertedblock2.10.10.19

-loading code version: csw1000block01.04.25.19
N = 56 subjects


In [5]:
## threshold based on filler questions
THRESHOLD = 0.5
plt_group_tqdf = get_thresholded_tqdf(group_tqdf,group_fqdf,threshold=THRESHOLD)
## remove begin node probe (only happens during training)
plt_group_tqdf = select_transition_probes(plt_group_tqdf)

# plotting

### group avg

In [6]:
num_subs = len(plt_group_tqdf.index.get_level_values('subjnum').unique())

# plotting
grouping = 'story'
M = plt_group_tqdf.correct_response.groupby([grouping]).mean()
N = plt_group_tqdf.correct_response.groupby([grouping]).count()
S = plt_group_tqdf.correct_response.groupby([grouping]).std()/np.sqrt(N)

plt.figure(figsize=(15,4))
plt.plot(np.arange(len(M))+1,M,c='purple',lw=3)
# plt.bar(np.arange(len(M))+1,M,yerr=S)

plt.axhline(0.5,c='lime',ls='--')
plt.yticks(np.arange(0,1.1,.1))
plt.ylim(0.2,1.01)
plt.grid(axis='y')

# single subject plotting
* NB because questions were asked on a random subset of trials, any given subject might not have answered questions on every trial. so cannot compute single subject/trial level metrics (e.g. acc or score)

In [7]:
for MA_WINDOW in [5]:
  subj_data = plt_group_tqdf.correct_response.groupby(['subjnum'])
  subj_data_plt = subj_data.apply(mov_avg,MA_WINDOW)

  plt.close('all')
  fig,axarr = plt.subplots(10,6,figsize=(36,30));axarr=axarr.reshape(-1)
  for plt_idx,sub_idx in enumerate(subj_data_plt.index):
    ax = axarr[plt_idx]
    ax.set_title(sub_idx)
    sub_ma = subj_data_plt[sub_idx]
    ax.plot(range(len(sub_ma)),sub_ma,c='purple',lw=3)
    ax.axvline(int(len(sub_ma)*(4/5))-15)
    ax.set_xticks([])
    ax.set_ylim(-0.05,1.05)
    ax.axhline(0.5,c='r',ls='--')
    ax.axhline(1,c='b',ls='--')

  title = 'subs_%s_window_%i'%(condition,MA_WINDOW)
  plt.savefig('figures/2022/subject/'+title)